In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
bern_data_path = '/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/prehospital/intubation_trigger/data/not_intubated/outcome_data/GCSRegaStudieBernNichtIntubiertOutcomes.xlsx'
stgallen_data_path = '/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/prehospital/intubation_trigger/data/not_intubated/outcome_data/GCSRegaStudieStGallenNichtIntubiertOutcomes.xlsx'
zuerich_data_path = '/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/prehospital/intubation_trigger/data/not_intubated/outcome_data/GCSRegaStudieZürichNichtIntubiertOutcomes.xlsx'
patient_id_path = '/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/prehospital/intubation_trigger/data/not_intubated/einsatznummern_GCS3_8.xlsx'

In [ ]:
bern_df = pd.read_excel(bern_data_path, header=5, sheet_name='Transportiert')
stgallen_df = pd.read_excel(stgallen_data_path, header=5, sheet_name='Transportiert')
zuerich_df = pd.read_excel(zuerich_data_path, header=5, sheet_name='Transportiert')
patient_id_df = pd.read_excel(patient_id_path)

In [ ]:
bern_df.rename(columns={'Sex [M/F]': 'sex'}, inplace=True)
stgallen_df.rename(columns={'Sex': 'sex'}, inplace=True)
zuerich_df.rename(columns={'Geschlecht': 'sex'}, inplace=True)

In [ ]:
overall_df = pd.concat([bern_df, stgallen_df, zuerich_df])

In [ ]:
target_hospitals = [
    'Kantonsspital St. Gallen - Stao St. Gallen (KSSG)',
    'Inselspital Bern',
    'Schockraum USZ, Zürich',
    'Universitätsspital Zürich (USZ)'
]

In [ ]:
overall_df = overall_df[overall_df['Zielort (Zielort)'].isin(target_hospitals)]

In [ ]:
# exclude NACA 6
overall_df = overall_df[overall_df['NACA (Medizinische Daten)'] < 6]

In [ ]:
overall_df = overall_df[~overall_df['Atemwegbefund (Atemweg Befund)'].isin(['Intubiert', 'Intubiert; Aspiration', 'Aspiration; Intubiert', 'Intubiert; Atemwege frei', 'Verlegt; Intubiert',
       'Intubiert; Verlegt; Aspiration', 'Intubiert; Verlegt',
       'Aspiration; Verlegt; Intubiert', 'Tracheotomiert'])]

In [ ]:
overall_df['Atemwegbefund (Atemweg Befund)'].unique()

In [ ]:
non_intubated = ['Wendel; Güdel',
       'Wendel', np.nan, 'Güdel; Bereits vorhanden', 'Güdel']

In [ ]:
overall_df = overall_df[overall_df['Bezeichnung (Atemweg-Management)'].isin(non_intubated)]

In [ ]:
print('Number of missions with non intubated patients:', overall_df['SNZ Einsatz Nr.  (Einsatz)'].nunique())
print('Number of missions with non intubated patients and GCS < 9:', overall_df[overall_df['GCS (Erstbefund)'] < 9]['SNZ Einsatz Nr.  (Einsatz)'].nunique())

In [ ]:
# restrict to patients in the patient_id_df
print('Number of missions after restriction to mission numbers: ', overall_df[overall_df['SNZ Einsatz Nr.  (Einsatz)'].isin(patient_id_df['SNZ Einsatz Nr.  (Einsatz)'])]['SNZ Einsatz Nr.  (Einsatz)'].nunique())

In [ ]:
overall_df = overall_df[overall_df['SNZ Einsatz Nr.  (Einsatz)'].isin(patient_id_df['SNZ Einsatz Nr.  (Einsatz)'])]

In [ ]:
outcome_columns = ['ICU [d]',
       'Ventilator [d]', 'Hospital stay [d]', '28 d Survival [y/n]',
       'Spital Survival [y/n]', 'ICU Survival [y/n]',
       'ITN ass. Pneumonia [y/n]']

In [ ]:
# outcome_present should be 1 if any outcome column is not nan
overall_df['outcome_present'] = overall_df.apply(lambda x: 1 if not all(x[outcome_columns].isna()) else 0, axis=1)

In [ ]:
outcome_df = overall_df[overall_df['outcome_present'] == 1]
# filter all where ICU [d] is not a number
outcome_df = outcome_df[outcome_df['ICU [d]'].apply(np.isreal)]

In [ ]:
outcome_df['SNZ Einsatz Nr.  (Einsatz)'].nunique(), outcome_df.shape

In [ ]:
# outcome_df.to_csv(os.path.join(os.path.dirname(stgallen_data_path), 'non_intubated_comatose_outcome_df.csv'))